# Template pour avoir les stats globales d'une seule équipes  
### Imports

In [176]:
import pandas as pd
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

In [177]:
data = pd.read_csv('01_2425_BDS/RAW_BDS_2024_2025.csv').reset_index()
data = data.rename(columns={"index":"id_carte"})

### Pre pross  
* Ajout score T1 et T2
* Ajout Victoire/Loose carte
* Gestion colonnes T1 = BDS, T2 = NMY  
* Ajout col BO1/3/5
* Ajout Victoire/Loose match
* Ajout id match et id map


In [178]:
data.head()

,id_carte,Team 1,Team 2,Score,Map,T1 Atk,T1 Def,T2 Atk,T2 Def,Event,...,T1 RT Start,T1 OT Start,T1 ATK RT,T1 DEF RT,T1 ATK OT,T1 DEF OT,T2 ATK RT,T2 DEF RT,T2 ATK OT,T2 DEF OT
0,0,Team BDS,FaZe Clan,9 : 7,Chalet,Montagne,Valkyrie,Deimos,Mozzie,Six Invitational 2025,...,atk,atk,4.0,2.0,2.0,1.0,4.0,2.0,1.0,0.0
1,1,Team BDS,FaZe Clan,5 : 7,Consulate,Blitz,Clash,Dokkaebi,Valkyrie,Six Invitational 2025,...,def,NaN,2.0,3.0,NaN,NaN,3.0,4.0,NaN,NaN
2,2,Team BDS,FaZe Clan,5 : 7,Nighthaven Labs,Ace,Valkyrie,Deimos,Kaid,Six Invitational 2025,...,atk,NaN,1.0,4.0,NaN,NaN,2.0,5.0,NaN,NaN
3,3,Team BDS,FaZe Clan,4 : 7,Clubhouse,Maverick,Valkyrie,Deimos,Kaid,Six Invitational 2025,...,def,NaN,2.0,2.0,NaN,NaN,4.0,3.0,NaN,NaN
4,4,Team BDS,FaZe Clan,0 : 0,Skyscraper,NaN,NaN,NaN,NaN,Six Invitational 2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [179]:
#Scores
data['T1 Score'] = data['Score'].str[0]
data['T1 Score'] = np.where(data['T1 Score']=='-',0,data['T1 Score'])
data['T1 Score'] = data['T1 Score'].astype(int)

data['T2 Score'] = data['Score'].str[-1]
data['T2 Score'] = np.where(data['T2 Score']=='-',0,data['T2 Score'])
data['T2 Score'] = data['T2 Score'].astype(int)

data = data.drop(columns=['Score'], axis=1)

In [180]:
#resultat carte
data['result_map'] = np.where(data['T1 Score']>data['T2 Score'],'win','loose')

In [181]:
#Colonnes

t1 = data[data['Team 1']=='Team BDS']
t2 = data[data['Team 2']=='Team BDS']

col = t2.columns
col2 = []
for k in col:
    if k == 'Team 1':
        col2.append('Team 2')
    
    elif k == 'Team 2':
        col2.append('Team 1')

    elif 'T1' in k:
        add = 'T2'+k[2:]
        col2.append(add)

    elif 'T2' in k:
        add = 'T1'+k[2:]
        col2.append(add)
    
    else:
        col2.append(k)

t2.columns = col2
t2['T1 RT Start'] = np.where( t2['T2 RT Start']=='atk','def',
                             np.where(t2['T2 RT Start']=='def','atk',t2['T2 RT Start']) )
t2['T1 OT Start'] = np.where( t2['T2 OT Start']=='atk','def',
                             np.where(t2['T2 OT Start']=='def','atk',t2['T2 OT Start']) )
t2 = t2[t1.columns]

data = pd.concat([t1,t2]).sort_values(by='id_carte')


In [183]:
#BO1/3/5
dfbo = data.groupby(['Team 1','Team 2','Event','Match','Date'])['id_carte'].count().reset_index()
dfbo = dfbo.rename(columns={"id_carte":"BO"})
dfbo["BO"] = "BO"+dfbo['BO'].astype(str)

data = data.merge(dfbo,on=['Team 1','Team 2','Event','Match','Date'],how='left')


In [ ]:
#W/L BO


In [184]:
data

,id_carte,Team 1,Team 2,Map,T1 Atk,T1 Def,T2 Atk,T2 Def,Event,Match,...,T1 ATK OT,T1 DEF OT,T2 ATK RT,T2 DEF RT,T2 ATK OT,T2 DEF OT,T1 Score,T2 Score,result_map,BO
0,0,Team BDS,FaZe Clan,Chalet,Montagne,Valkyrie,Deimos,Mozzie,Six Invitational 2025,Grand Final,...,2.0,1.0,4.0,2.0,1.0,0.0,9,7,win,BO5
1,1,Team BDS,FaZe Clan,Consulate,Blitz,Clash,Dokkaebi,Valkyrie,Six Invitational 2025,Grand Final,...,NaN,NaN,3.0,4.0,NaN,NaN,5,7,loose,BO5
2,2,Team BDS,FaZe Clan,Nighthaven Labs,Ace,Valkyrie,Deimos,Kaid,Six Invitational 2025,Grand Final,...,NaN,NaN,2.0,5.0,NaN,NaN,5,7,loose,BO5
3,3,Team BDS,FaZe Clan,Clubhouse,Maverick,Valkyrie,Deimos,Kaid,Six Invitational 2025,Grand Final,...,NaN,NaN,4.0,3.0,NaN,NaN,4,7,loose,BO5
4,4,Team BDS,FaZe Clan,Skyscraper,NaN,NaN,NaN,NaN,Six Invitational 2025,Grand Final,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,loose,BO5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,126,Team BDS,G2 Esports,Oregon,NaN,NaN,NaN,NaN,Malta Cyber Series VII,Upper Bracket Final,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,loose,BO3
127,127,Team BDS,Team Secret,Skyscraper,Grim,Fenrir,Flores,Valkyrie,Malta Cyber Series VII,Upper Bracket Semifinals,...,NaN,NaN,4.0,3.0,NaN,NaN,5,7,win,BO3
128,128,Team BDS,Team Secret,Nighthaven Labs,Ying,Valkyrie,Dokkaebi,Fenrir,Malta Cyber Series VII,Upper Bracket Semifinals,...,NaN,NaN,2.0,2.0,NaN,NaN,7,4,loose,BO3
129,129,Team BDS,Team Secret,Consulate,Ying,Fenrir,Dokkaebi,Valkyrie,Malta Cyber Series VII,Upper Bracket Semifinals,...,1.0,1.0,1.0,5.0,0.0,0.0,8,6,loose,BO3
